In [9]:
import os
import pandas as pd
import soundfile as sf
from soundfile import LibsndfileError
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from ds_creation.download_utils import download_dataset
from ds_creation.ds_utility import species_spec
from ds_creation.plot_utility import process_audio_files, tsne_calc

# folder for metadata
md_path = os.path.join('data_cache', 'metadata')
if not os.path.exists(md_path):
    os.makedirs(md_path)

# folders for data
ds_dir = 'data'
audio_dir = 'audio'
spec_dir = 'mammals_calls_grey'

data_path = os.path.join(ds_dir, audio_dir)
if not os.path.exists(data_path):
    os.makedirs(data_path)
    
spec_path = os.path.join(ds_dir, spec_dir)
if not os.path.exists(spec_path):
    os.makedirs(spec_path)

npy_path = os.path.join(ds_dir, 'npy_files')
if not os.path.exists(npy_path):
    os.makedirs(npy_path)   

DOWNLOAD_FILES = False
url = 'https://whoicf2.whoi.edu/science/B/whalesounds/fullCuts.cfm'

MODELS_METRICS_DIR = 'models_metrics'
if not os.path.exists(MODELS_METRICS_DIR):
    os.makedirs(MODELS_METRICS_DIR)


## Download dataset

In [10]:
if DOWNLOAD_FILES:
    download_dataset(data_path, url, 'getSpecies', 'pickYear')

In [11]:
# Create metadata file with species and file counts
folder_list = []
for folder in os.listdir(data_path):
    folder_path = os.path.join(data_path, folder)
    print(f'Processing species {folder}')
    if os.path.isdir(folder_path):
        for file in os.listdir(folder_path):
            if file.endswith('.wav'):
                file_path = os.path.join(folder_path, file)
                try:
                    f = sf.SoundFile(file_path)
                    duration = int(f.frames / f.samplerate)
                    f.close()
                except LibsndfileError as e:
                    print(f'Error reading {file_path}: {e}')
                    continue

                folder_list.append({'species': folder, 'filename': file, 'duration_sec': duration})


species_df = pd.DataFrame(folder_list)
species_df.to_csv(os.path.join(md_path, 'files_len.csv'), index=False)

Nel file 'files_len.csv' sono presenti tutte le durate dei singoli file audio in secondi arrotondati per difetto, quindi vado a contare quanti ritagli da 2 secondi riesco ad ottenere per ogni specie con la sliding window con hop di 1 secondo.

In [12]:
md_ds = pd.read_csv(os.path.join(md_path, 'files_len.csv'))
md_ds = md_ds.dropna(subset=['duration_sec'])
long_md_ds = md_ds[md_ds['duration_sec'] > 1]

EmptyDataError: No columns to parse from file

In [ ]:
# dataframe with all audio and dataframe with audio longer than 1 second
tot_audio_df = md_ds.drop(columns=['filename'])
long_audio_df = long_md_ds.drop(columns=['filename'])

# sum of audio lengths by species
sum_audio_len = tot_audio_df.groupby('species').agg(all_audio_len = ('duration_sec', 'sum'))
sum_long_audio_len = long_audio_df.groupby('species').agg(filtered_audio_len = ('duration_sec', 'sum'))

# merge dataframes and calculate the number of seconds lost by filtering out audio shorter than 2 seconds
sum_audio_len['filtered_audio_len'] = sum_long_audio_len['filtered_audio_len']
sum_audio_len['difference'] = sum_audio_len['all_audio_len'] - sum_audio_len['filtered_audio_len']

print(sum_audio_len.sort_values(by='difference', ascending=False))

## Creazione spettrogrammi

In [ ]:
FRAME_SIZE = 1024
HOP_SIZE = 512
OUTPUT_DIR = spec_path
CHUNK_SIZE = 2

In [ ]:
tree = os.walk(os.path.join(data_path))
root, dirs, files = next(tree)
unpopular_species_list = md_ds['species'].unique()

stf_config = {'frame_win': FRAME_SIZE, 'hop': HOP_SIZE, 'fs': FRAME_SIZE}
spec_config = {'size': CHUNK_SIZE, 'hop': CHUNK_SIZE}

output_dir = {'spec': spec_path, 'num': npy_path}

species_spec(dirs, unpopular_species_list, output_dir, data_path, stf_config, spec_config, 'gray')

## Calcolo del MFCC

In [ ]:
audio_df_exploded, mfcc_matrix = process_audio_files(species_df['species'].tolist(), data_path, MODELS_METRICS_DIR)
print(f'Different species in audio dataset: {audio_df_exploded["species"].unique()}')
print(audio_df_exploded.head())


In [ ]:
tsne_df = tsne_calc(audio_df_exploded, mfcc_matrix, MODELS_METRICS_DIR)